<a href="https://colab.research.google.com/github/MiroGasparek/DeepLearningCourse/blob/master/keras_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

